In [1]:
#load the packages
import polars as pl
#make sure that this is the main file
import sys
import os
project_root = os.getcwd()
sys.path.insert(0, project_root)

### Training Dataset


Name your data as Train.csv and place it in the Training Data folder. Or you can change the path in the code below.


In [2]:
# keep you training dataset in the training data folder
# this template uses csv files 
# column names can be set in Python but this template does not automatically update the column for the demo 
# however, the function will give you the option to tell column names for the text and label data

df_train = pl.read_csv("Training Data/Train.csv",encoding='ISO-8859-1') 
print(f"Dataset shape: {df_train.shape[0]} rows and {df_train.shape[1]} columns")


Dataset shape: 162758 rows and 5 columns


### DEMO

In [3]:
df_train.head()
# randomly select only 10% of the data since the dataset is large
#RUN ONLY ONCE
df_train = df_train.sample(fraction=0.1, shuffle=True, seed=42) 
df_train

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
str,str,bool,str,str
"""john_mcclane_rocky_balboa_laby…","""Deborah Farley""",false,"""..a cocksure, stylized, gutty …","""POSITIVE"""
"""tyler_durden_dazzling_incredib…","""Margaret Martin""",false,"""A wild ride of a movie, Spring…","""POSITIVE"""
"""glimmer_mythical_han_solo""","""Angel Peters""",false,"""In a year of many car movies, …","""POSITIVE"""
"""gandalf_journey_epic""","""Alexandria Wilson""",true,"""Festival will surely tickle yo…","""POSITIVE"""
"""vito_corleone_lara_croft_travi…","""Andrew Blankenship""",false,"""The limp&#44; negligible resul…","""NEGATIVE"""
…,…,…,…,…
"""golden_adventure_holly_golight…","""Roberta Alexander""",false,"""It may not send shivers up any…","""POSITIVE"""
"""the_bride_zephyr_mystique""","""Karen Adams""",false,"""Steven Spielberg's new film is…","""POSITIVE"""
"""destiny_valiant_dream""","""Nicholas Reynolds""",false,"""It's derivative as hell, but t…","""NEGATIVE"""


The dataset is for training. The sentiments are already labeled. This will allow us to train a model that can predict sentiments on new data.


In [4]:
# here I have three python script I built to pre_process the data and running the pipeline
# you can find the code in the tools/preprocess.py file
# you can find  the code in the tools/pipeline.py file
# the pre_process function is used to clean the text data, there are various options available, please check the tools/preprocess.py file for details
# the run_pipeline function is used to run the sentimental analysis pipeline, it takes the training data and the vectorizer and machine learning methods as input, and returns the results
from tools.preprocess import pre_process
#this function will run the sentimental analysis in the training data and return the results
from tools.pipeline import run_pipeline
# this function will run the sentimental analysis in the new data and return the predictions
from tools.predict import make_predictions



NLTK data 'punkt' already present.
NLTK data 'stopwords' already present.
NLTK data 'wordnet' downloaded.
NLTK data 'omw-1.4' downloaded.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\meala\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
# you can use the pre_process function to clean the text data
response_column = "reviewText" # this is the column name for the text data, feel free to change it to your text column name
sentiment_column = "sentiment" # this is the column name for the sentiment data, feel free to change it to your sentiment column name
print(df_train[response_column][2])
print("\n" + pre_process(df_train[response_column][2]) )

In a year of many car movies, this one was the best (Living Life Fearless)

year many car movie one best living life fearless


In [6]:
# make changes as necessary
# inside the map_elements, add  the parameters [pre_process(x, parameters_to_be_added)] and set it True/False if it differs from the defualt value
# check the tools/preprocess.py file for the parameters and their default values
# some of the parameters are remove_brackets, remove_stopwords, remove_punctuation, remove_numbers, remove_emojis, remove_urls, remove_html_tags, lemmatize, stem, lowercase
df_train = df_train.with_columns(
    pl.col(response_column).map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
df_train.head(10)

C:\Users\meala\AppData\Local\Temp\ipykernel_21836\4018690164.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_train = df_train.with_columns(


movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,processed
str,str,bool,str,str,str
"""john_mcclane_rocky_balboa_laby…","""Deborah Farley""",false,"""..a cocksure, stylized, gutty …","""POSITIVE""","""cocksure stylized gutty thrill…"
"""tyler_durden_dazzling_incredib…","""Margaret Martin""",false,"""A wild ride of a movie, Spring…","""POSITIVE""","""wild ride movie spring breaker…"
"""glimmer_mythical_han_solo""","""Angel Peters""",false,"""In a year of many car movies, …","""POSITIVE""","""year many car movie one best l…"
"""gandalf_journey_epic""","""Alexandria Wilson""",true,"""Festival will surely tickle yo…","""POSITIVE""","""festival surely tickle funny b…"
"""vito_corleone_lara_croft_travi…","""Andrew Blankenship""",false,"""The limp&#44; negligible resul…","""NEGATIVE""","""limp 44 negligible result unwe…"
"""zephyr_scarlett_o'hara_magicia…","""Lee Griffin""",false,"""The film does have it's weak s…","""POSITIVE""","""film weak spot part dead see"""
"""the_joker_quest_rocky_balboa""","""Brianna Flores""",true,"""A good-hearted entertainment t…","""POSITIVE""","""goodhearted entertainment mana…"
"""surreal_the_captain_america_go…","""Shelia Miller""",false,"""At this moment, 500 Days of Su…","""POSITIVE""","""moment 500 day summer favorite…"
"""dorothy_gale_gollum_frodo_bagg…","""Chelsea Martinez""",false,"""Beats thrums with an unbridled…","""POSITIVE""","""beat thrum unbridled energy ow…"


In [ ]:
#### in this template, there are four text representation / vectorizer methods available 
#### in the function run_pipeline (in python cell below), we shall make use of this, write the words inside [ ] for the methods you want to use
#### 1. Bag of Words [BOW] 
#### 2. Term Frequency [tf]
#### 3. TF -IDF    [tfidf]
#### 4. Word Embedding using Word2Vec (you can use other packages with slight changes) [wv] 
         # Word Embedding uses defualt 300 values; this will take some time to run

In [ ]:
#### in this template, there are also three machine learning methods that can be used
#### 1. Logistic Regression [logit]
#### 2. Random forest (recommended) (rf)
#### 3. XGBoosting  [XGB](word embedding and XGBoost may take long time to complete, combination of both is not recommended in local machine)

#I will keep this repository updated, and I will add more methods in the future

In [7]:
# this is the example of how to use the function
# you can change the vectorizer_name and model_name to the ones you want to use
# for now we will use word embedding and logistic regression
# write the name of your columns in the text_column_name and sentiment_column_name
# the text_column_name is the column name of the text data, and sentiment_column_name is

# run_pipeline function will return the dataframe with the vectorized text, vectorizer used  and the model
# it will also print the results of the model, including the accuracy and F1 score
# note, even without hyperparameter tuning, the model is getting over 70% accuracy in my test
# there may not be a need to perform hyperparameter tuning, but you can set perform_tuning to True if you want to do that

dt= run_pipeline(
    vectorizer_name="wv", # BOW, tf, tfidf, wv
    model_name="logit", # logit, rf, XGB .#XGB takes long time, can not recommend using it on normal case
    df=df_train,
    text_column_name="processed",  # this is the column name of the text data, 
    sentiment_column_name = "sentiment",
    perform_tuning = True # make this true if you want to perform hyperparameter tuning, it will take longer time and 
                            # may run out of memory if the dataset is large,
)

# missing values in the text data will be removed

--- Running Pipeline for Wv + Logit ---
Labels encoded: Original -> ['NEGATIVE' 'POSITIVE'], Encoded -> [0 1]
1. Vectorizing entire dataset (X)...
Loading pre-trained word2vec-google-news-300 model (this may take a few minutes)...
Word2Vec model loaded.
2. Splitting data into train/test...
3. Training and predicting...
   - Starting Logistic Regression training with GridSearchCV for hyperparameter tuning...
   - Using default parameter grid for tuning: {'solver': ['liblinear', 'lbfgs'], 'C': [0.1, 1.0, 10.0], 'class_weight': [None, 'balanced'], 'max_iter': [500, 1000]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits

   - Best Hyperparameters found:
{'C': 10.0, 'class_weight': None, 'max_iter': 500, 'solver': 'lbfgs'}
   - Best Cross-Validation Score (F1-weighted): 0.7597
Best model parameters: {'C': 10.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'deprecated', 'n_jobs': None, 'pen

In [8]:
## the dt is a dictionary that contains the results of the model, including the accuracy and F1 score
print(dt.keys())
# you can access the results using the keys of the dictionary
print("Vectorizer used: ", dt["vectorizer_name"])
print("Model used: ", dt["model_object"])
print("Accuracy: ", dt["accuracy"])



dict_keys(['model_object', 'vectorizer_name', 'vectorizer_object', 'label_encoder', 'y_test', 'y_pred', 'accuracy', 'report'])
Vectorizer used:  wv
Model used:  LogisticRegression(C=10.0, max_iter=500, random_state=42)
Accuracy:  0.76896


### New Dataset for prediction
You can use the same format as the training dataset, but ensure that it contains the "Response" column for text data. The "Sentiment" column is optional for prediction datasets, as it will be generated by the model.
Make sure the dataset is saved in the "New Data" folder and is in CSV format.

In [9]:
new_data = pl.read_csv("New Data/test.csv",encoding='ISO-8859-1') #keep your file here
print(new_data.shape)
new_data= new_data.sample(fraction=0.25, shuffle=True, seed=42)
print(new_data.shape)

(55315, 4)
(13828, 4)


In [10]:
new_data = new_data.with_columns(
    pl.col(response_column).map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
new_data.head(10)

C:\Users\meala\AppData\Local\Temp\ipykernel_21836\3427585582.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  new_data = new_data.with_columns(


movieid,reviewerName,isTopCritic,reviewText,processed
str,str,bool,str,str
"""frodo_baggins_rocky_balboa_she…","""Toni Vaughn""",false,"""The result is an unsettling ta…","""result unsettling tale human p…"
"""stardust_john_mcclane""","""Carol Jennings""",false,"""Think twice about getting invo…","""think twice getting involved w…"
"""hermione_granger_sherlock_holm…","""Tara Huang""",true,"""A film that's so bloody wonder…","""film bloody wonderful meaning …"
"""astonish_valiant""","""Shelley Murillo""",false,"""...a decent setup that's emplo…","""decent setup employed progress…"
"""indiana_jones_dazzling_dorothy…","""Daniel Bond""",false,"""Inspiring? Not to me. Lamentab…","""inspiring lamentably bland exe…"
"""glimmer_hannibal_lecter_frodo_…","""Mrs. Nicole Fleming""",false,"""This perfectly executed piece …","""perfectly executed piece movie…"
"""brave_anakin_skywalker""","""Melissa Harrington""",false,"""The Richard Curtis script has …","""richard curtis script rooney m…"
"""katniss_everdeen_superman""","""Mckenzie Ortiz""",false,"""A fun-filled afternoon of dino…","""funfilled afternoon dinotastic…"
"""secret_magic_john_wick_legend""","""Samantha Ware""",false,"""I'm still not sure what this i…","""still sure supposed save bunch…"


In [11]:
make_predictions(
    new_data=new_data,
    text_column_name="processed",
    vectorizer=dt["vectorizer_object"],
    best_model=dt["model_object"],
    label_encoder=dt["label_encoder"],
    prediction_column_name="sentiment_predictions"  # Optional custom name
)

movieid,reviewerName,isTopCritic,reviewText,processed,sentiment_predictions
str,str,bool,str,str,str
"""frodo_baggins_rocky_balboa_she…","""Toni Vaughn""",false,"""The result is an unsettling ta…","""result unsettling tale human p…","""POSITIVE"""
"""stardust_john_mcclane""","""Carol Jennings""",false,"""Think twice about getting invo…","""think twice getting involved w…","""NEGATIVE"""
"""hermione_granger_sherlock_holm…","""Tara Huang""",true,"""A film that's so bloody wonder…","""film bloody wonderful meaning …","""POSITIVE"""
"""astonish_valiant""","""Shelley Murillo""",false,"""...a decent setup that's emplo…","""decent setup employed progress…","""NEGATIVE"""
"""indiana_jones_dazzling_dorothy…","""Daniel Bond""",false,"""Inspiring? Not to me. Lamentab…","""inspiring lamentably bland exe…","""NEGATIVE"""
…,…,…,…,…,…
"""vivid_intrigue_celestial""","""Luke Reyes""",false,"""This is the film's ultimate me…","""film ultimate message nt think…","""POSITIVE"""
"""mystery_lost_james_t._kirk_sta…","""Kathy Wade""",false,"""It's the best Sondheim adaptat…","""best sondheim adaptation sayin…","""POSITIVE"""
"""hiccup_terminator_myriad_capta…","""Troy Watson""",false,"""The South Australian actor her…","""south australian actor deliver…","""POSITIVE"""
